# VacationPy
----
The following displays the following using google's maps javascript api (https://developers.google.com/maps/documentation/javascript):

*the humidity (%) of cities considered in the weather analysis displayed as a heat map; and

*hotels in cities with ideal weather conditions 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame


In [2]:
#load data

input_path = "../output_data/cities.csv"

data =pd.read_csv(input_path, delimiter=",", index_col="City_ID")

data

,City,Lat,Lng,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Datetime
City_ID,,,,,,,,,
0,Albany,42.60,-73.97,79.00,66,12,3.04,US,1592577607
1,Jamestown,42.10,-79.24,75.00,73,40,4.70,US,1592577743
2,Mataura,-46.19,168.86,39.00,83,100,3.00,NZ,1592577743
3,Barrow,71.29,-156.79,35.60,93,90,12.75,US,1592577743
4,Klaksvík,62.23,-6.59,57.20,71,84,9.17,FO,1592577744
...,...,...,...,...,...,...,...,...,...
565,Port Shepstone,-30.74,30.45,66.47,71,0,18.39,ZA,1592578811
566,Lhokseumawe,5.18,97.15,80.06,78,85,4.52,ID,1592578787
567,Corvallis,44.56,-123.26,64.99,100,1,1.86,US,1592578701


### Humidity Heatmap

>Configure gmaps.

>Use the Lat and Lng as locations and Humidity as the weight.

>Add Heatmap layer to map.

In [3]:
#setup 

    #configure gmaps
gmaps.configure(api_key = g_key)

coord = data[["Lat", "Lng"]]

weights = data["Humidity (%)"]

    #format display
figure_layout = {
    "width": '500px',
    "height": '500px',
    "border": '1px solid black',
    "padding": '1px',
    "margin": '0 auto 0 auto'}


# set max intensity
    #create and display fig
hum_fig = gmaps.figure(layout=figure_layout)
heatmap_layer = gmaps.heatmap_layer(coord, weights=weights)
hum_fig.add_layer(heatmap_layer)

#set max intensity
heatmap_layer.max_intensity = data["Humidity (%)"].max()


#display
hum_fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria

>Narrow down the cities to fit weather conditions.

>Drop any rows will null values.

In [4]:
#select data: A max temperature lower than 80 degrees but higher than 70; 
#wind speed of less than 10 mph; and humidity less than 50%

narrowed_data = data.loc[(data["Max Temperature (F)"]>68) &
                         (data["Max Temperature (F)"]<80) &
                         (data["Humidity (%)"]<50) &
                         (data["Cloudiness (%)"]<10),:]

#identify null values

print(f'\nno. of cities in original dataset: {len(narrowed_data["City"])}')

    #identify null rows: unique = 2

null = narrowed_data.notnull()

    #identify rows with nulls
IDs = null.loc[null["Country"]==False, :].index.tolist()

    #drop rows with nulls
cleaned_data= narrowed_data.drop(IDs, inplace=False)

    #verify
print(f'\nno. of cities in cleaned dataset: {len(cleaned_data["City"])}')


no. of cities in original dataset: 17

no. of cities in cleaned dataset: 15


### Hotel Map

>Store into variable named hotel_df.

>Add a "Hotel Name" column to the DataFrame.

>Set parameters to search for hotels with 5000 meters.

>Hit the Google Places API for each city's coordinates.

>Store the first Hotel result into the DataFrame.

>Plot markers on top of the heatmap.

In [5]:
#setup list

name = []

# set up a parameters

for index, row in cleaned_data.iterrows():
    
    parameters = {"location": f'{row["Lat"]},{row["Lng"]}',
                    "keyword":"hotel",
                    "radius": 5000,
                    "type": "establishment",
                    "key": g_key}

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=parameters)
    
    #append name to list
    try:
        name.append(response.json()["results"][0]["name"])
    except:
        name.append("no hotel")
    

In [6]:
#verify output
len(name)

15

In [7]:
#merge hotels back to original df
cleaned_data["Hotel"] = name

#remove cities without hotels
hotel_df = cleaned_data.loc[cleaned_data["Hotel"] != "no hotel", :]

#verify that there are not nulls in the dataset: unique =1
if sum(hotel_df.notnull().nunique()) ==10:
    print("no duplicates")

else:
    print("remove duplicates before proceeding")

no duplicates


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

hum_fig.add_layer(gmaps.marker_layer(locations, info_box_content=hotel_info))

# Display figure
hum_fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…